<table width="100%"><tr><td style="color:#bbbbbb;background-color:#ffffff;font-size:11px;font-style:italic;text-align:right;">LaTeX macros. Do not delete!</td></tr></table>
$ \newcommand{\bra}[1]{\langle #1|} $
$ \newcommand{\ket}[1]{|#1\rangle} $
$ \newcommand{\braket}[2]{\langle #1|#2\rangle} $
$ \newcommand{\mymatrix}[2]{\left( \begin{array}{#1} #2\end{array} \right)} $

![Constantine Quantum Technologies for QFF22-Algiers](imgs/qff22_cqtech_banner.png)

# Quantum Key Distribution Workshop | Qiskit Fall Fest 2022, Algiers

## 1. Introduction to Quantum Key Distribution, BB84

Nowadays, nearly everyone is using some form of encryption to store, send, or receive data. If you own a phone, there's a high chance that you have set a password to prevent unauthorized persons from accessing your private files and messages. These messages are themeselves sometimes encrypted when you hit the send button, and decrypted when they get to your contact's phone.<br>
But to ensure that you and your contact can properly carry out this operation, you must share some sort of secret key that allows you to communicate in a secure way. Therefor, you must at some point share this key through some communication channel. Do you see the issue here? If someone were to listen to that channel when you were sharing that key, they would be able to decrypt all of your transmissions!

Quantum Key Distribution (QKD) aims at providing a way that makes it impossible for a third person, which we will call *Eve*, to gain knowledge about your key without alerting the two communicating persons, *Alice* and *Bob*.

Two properties of quantum states, as opposed to classical states, will be useful to achieve this purpose: the wave function collapse at the measurement, and the no-cloning theorem.

#### Wave function collapse
In the case of quantum binary information, qubits can exist in a superposition of two fundamental states. This is unlike the classical bits which can either be in the states $0$ or $1$.<br>

The general state of a qubit is written as

$$\ket{q} = \alpha\ket{0} + \beta\ket{1},$$

where $\ket{q}$ is said to be in a superposition of the states $0$ and $1$. However, if we observe or measure this qubit, then we will only be able to see it in one of the states, with probabilities $|\alpha|^2$ and $|\beta|^2$ respectively.

Most importantly, after we observe it to be in either of these states, $\ket{q}$ will then no longer be in a superposition state but will *collapse* to the observed state. That is, if we observe it to be $0$, then $\ket{q} = \ket{0}$, and if we observe $1$, then $\ket{q} = \ket{1}$. The state $\ket{q}$ is not in a superposition anymore.

Therefor, if Alice sends a quantum message to Bob, and Eve measures it before it gets to him, Bob **will not** get the original message sent by Alice! He will get a version where all of the qubits of the message are collapsed into $\ket{0}$ or $\ket{1}$.

#### No-clone theorem
Without getting into details that are outside the scope of this workshop, quantum mechanics tells us that unlike a classical state, quantum states **cannot** be copied. You cannot take a qubit and make a copy of it, as this will violate the laws of physics.

Although the justification of such a theorem is of little use to us, one of its consequences is essential to QKD: if we were to send a key not as a classical bitstring but as a quantum message, a third person will never be able to make a copy of it!

### The BB84 QKD Protocol
Proposed by Charles Bennett and Gilles Brassard in 1984, thus the name BB84, the BB84 protocol is a scheme that Alice and Bob can use to safely share a key using the two properties of quantum mechanics described above. This protocol makes it impossible for Eve to intercept the key without alerting Alice and Bob.<br>
The protocol is achieved by follow a series of simple steps:

1. Alice generates a random bitstring (zeros and ones) and converts it to a series of qubits correspondingly. The zeros and ones can either be encoded as $\ket{0}$ and $\ket{1}$, or the superposition states $\ket{+}$ and $\ket{-}$. In the first case Alice has chosen the **z-basis**, and in the second choice she has chosen the **x-basis**, where $\ket{+}$ is a $0$ and $\ket{-}$ is a $1$. For each bit, she chooses a basis randomly. She then sends everything to Bob.
1. Bob measures the received message, measuring each qubit in a random basis (x or z).
1. Alice and Bob then make their choice of bases public. They will keep each bit where they used the same basis, and discard the rest. The final bitstring they end up with is the key.
1. To make sure that no problems occured during the protocol, Alice and Bob will each reveal a random small sample of predetermined bits from their keys and compare the two versions. If the process was successful, the samples will be the same, otherwise, there will be some differences.

## 2. Implementation of BB84

*(The following codes were taken and modified from [Qiskit's tutorials](https://qiskit.org/textbook/ch-algorithms/quantum-key-distribution.html))*

We first start by importing the relevent libraries and functions.

In [10]:
from qiskit import QuantumCircuit, Aer, transpile, assemble
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from numpy.random import randint
import numpy as np

### Step 1

In the first step of BB84, Alice sends a series of quantum states to Bob. She chooses a random list of bits, and a random set of bases, and then encodes each bit in its corresponding basis:

**The Z-Basis:** where 0 is encoded as $\ket{0}$ and 1 is encoded as $\ket{1}$.<br>
**The X-Basis:** where 0 is encoded as $\frac{1}{\sqrt{2}}(\ket{0}+\ket{1})$ and 1 is encoded as $\frac{1}{\sqrt{2}}(\ket{0}-\ket{1})$.

Recall that the operation that transforms the Z-Basis to the X-Basis and vice versa is the Hadamard gate:

$$H\ket{0}=\ket{+}=\frac{1}{\sqrt{2}}(\ket{0}+\ket{1})$$
$$H\ket{1}=\ket{-}=\frac{1}{\sqrt{2}}(\ket{0}-\ket{1})$$
and
$$H\ket{+}=H\frac{1}{\sqrt{2}}(\ket{0}+\ket{1})=\ket{0}$$
$$H\ket{-}= H\frac{1}{\sqrt{2}}(\ket{0}-\ket{1})=\ket{1}$$

So we define a function *encode_message(bits, bases)*, where *bits* and *bases* are bitstrings, which does just that:<br>
- For each bit in the list *bits*, we start with a quantum circuit containing a single state $\ket{0}$, which is the default initial state.
- If the bit is $1$, we change that state to $\ket{1}$ by applying a NOT gate.
- Then we look at its corresponding basis; if it's $0$ (Z-basis), we keep the previous state, if it's $1$ (X-basis), we apply a Hadamard gate on the state and get the resulting state $\ket{+}$ or $\ket{-}$.

At the end, we end up with a random message of 0s and 1s, encoded randomly in the Z or X bases:

$$Bits =  [0, 0, 1, 0, 1, 1, 1, 0, ...]$$
$$Bases =  [Z, X, X, Z, Z, Z, X, X, ...]$$

So the message will look like this:
$$M = [\ket{0}, \ket{+}, \ket{-}, \ket{0}, \ket{1}, \ket{1}, \ket{-}, \ket{+}, ...]$$

Alice then sends this *quantum message* to Bob, most probably using photons, where each photon encodes a quantum state.

In the cell bellow, we implement such a function.

In [11]:
def encode_message(bits, bases):
    message = []
    for i in range(n):
        qc = QuantumCircuit(1,1)
        if bases[i] == 0: # Prepare qubit in Z-basis
            if bits[i] == 0:
                pass 
            else:
                qc.x(0)
        else: # Prepare qubit in X-basis
            if bits[i] == 0:
                qc.h(0)
            else:
                qc.x(0)
                qc.h(0)
        message.append(qc)
    return message

### Step 2

In the second step, Bob measures the quantum message he received form Alice, using a random set of bases of his own.<br>
So he starts by choosing his bases, then applies the basis transformations where needed (Hadamard gates), and measures the received qubits.

For example, if Bob receives the message

$$M = [\ket{0}, \ket{+}, \ket{-}, \ket{0}, \ket{1}, \ket{1}, \ket{-}, \ket{+}, ...]$$

And chooses the bases

$$Bases =  [Z, Z, X, X, Z, Z, Z, X, ...]$$

Then his measurements will yield

$$Bits =  [0, 0/1, 1, 0/1, 1, 1, 0/1, 0, ...]$$

The following function implements this process, which can be thought of as the reverse process that Alice perfoms in step 1, but with different bases.

In [12]:
def measure_message(message, bases):
    measurements = []
    
    for q in range(n):
        if bases[q] == 0: # measuring in Z-basis
            message[q].measure(0,0)
        if bases[q] == 1: # measuring in X-basis
            message[q].h(0)
            message[q].measure(0,0)
        
        # This part is a Qiskit technicality, don't worry about it.    
        aer_sim = Aer.get_backend('aer_simulator')
        qobj = assemble(message[q], shots=1, memory=True)
        result = aer_sim.run(qobj).result()
        
        measured_bit = int(result.get_memory()[0])
        measurements.append(measured_bit)
    return measurements

### Step 3

After Bob makes his measurements, he and Alice make their bases public. After which, they will both keep the bits where they used the same bases, and discard the rest.

In the previous steps, the bases used were

$$Bases_{Alice} =  [Z, X, X, Z, Z, Z, X, X, ...]$$
$$Bases_{Bob}   =  [Z, Z, X, X, Z, Z, Z, X, ...]$$

So they will discard their second, fourth, seventh bits, and so on.

Therefor, the remaining bits are ensured to be exactly the same for both Alice and Bob **IF** the message sent by Alice has not been meddled with.

$$Bits_{final}   =  [0, 1, 1, 1, 0, ...]$$

The implementation is straightforward.

In [13]:
def remove_garbage(a_bases, b_bases, bits):
    good_bits = []
    for q in range(n):
        if a_bases[q] == b_bases[q]:
            # If both used the same basis, add
            # this to the list of 'good' bits
            good_bits.append(bits[q])
    return good_bits

 ### Step 4
 
To make sure that both Alice and Bob have the same bits/key, they compare a random sample of their bits. If everything went well, their samples will be the same. If something went wrong in the previous steps, the samples will be different and they will discrad the generated key.<br>
An example of something going wrong is an eavesdropper, called Eve, measuring the quantum message before it reaches Bob. This will have the effect of collapsing all the quantum states to either $\ket{0}$ or $\ket{1}$, rendering the message received by Bob **different** from the one sent by Alice.

Here, we only implement the sampling function.

In [14]:
def sample_bits(bits, selection):
    sample = []
    for i in selection:
        # use np.mod to make sure the
        # bit we sample is always in 
        # the list range
        i = np.mod(i, len(bits))
        # pop(i) removes the element of the
        # list at index 'i'
        sample.append(bits.pop(i))
    return sample

### BB84 without interception: full code

Having described and implemented the steps for BB84, we can now write the complete code for the protocol.

In the following code, we showcase the ideal scenario where everything goes as planned. At the end, we compare the samples generated by Alice and Bob observe that no error occured.

In [21]:
np.random.seed(seed=3)  # We use a known seed for RNG to make the results reproducible.
n = 100

## Step 1: Alice generates random bits.
alice_bits = randint(2, size=n)
## Step 2: Alice chooses a series of random bases: one for each bit.
alice_bases = randint(2, size=n)

## Step 3: Alice then sends a quantum message with her bits encoded in her random bases.
message = encode_message(alice_bits, alice_bases)

## Step 4: Bob chooses random bases of his own.
bob_bases = randint(2, size=n)
## Step 5: Bob then measures Alice's message in his own bases.
bob_results = measure_message(message, bob_bases)

## Step 6: Alice and Bob make their bases public, compare them, and only keep
#          the measurements where they used the same bases.
bob_key = remove_garbage(alice_bases, bob_bases, bob_results)
alice_key = remove_garbage(alice_bases, bob_bases, alice_bits)


## Step 5
sample_size = 15
bit_selection = randint(n, size=sample_size)

bob_sample = sample_bits(bob_key, bit_selection)
print("  bob_sample = " + str(bob_sample))
alice_sample = sample_bits(alice_key, bit_selection)
print("alice_sample = "+ str(alice_sample))

diff_sample = [0 if a == b else 1 for a, b in zip(alice_sample, bob_sample)]
print("      errors = "+ str(diff_sample))

print("-----------")
print("  bob_key = " + str(bob_key))
print("alice_key = "+ str(alice_key))


  bob_sample = [0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1]
alice_sample = [0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1]
      errors = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
-----------
  bob_key = [0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1]
alice_key = [0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1]


### BB84 with interception: full code 

Now let's consider the case where Eve intercepts the message sent by Alice and tries to measure it. This is simulated between steps 3 and 4, where Eve measures the quantum message in a random set of bases.

The rest of the code is exactly the same. However, when Alice and Bob compare their samples, they will find multiple differences! This will alert them of Eve's interception.

In [22]:
np.random.seed(seed=3)  # We use a known seed for RNG to make the results reproducible.
n = 100

## Step 1: Alice generates random bits.
alice_bits = randint(2, size=n)
## Step 2: Alice chooses a series of random bases: one for each bit.
alice_bases = randint(2, size=n)

## Step 3: Alice then sends a quantum message with her bits encoded in her random bases.
message = encode_message(alice_bits, alice_bases)

## Interception
eve_bases = randint(2, size=n)
eve_results = measure_message(message, eve_bases)

## Step 4: Bob chooses random bases of his own.
bob_bases = randint(2, size=n)
## Step 5: Bob then measures Alice's message in his own bases.
bob_results = measure_message(message, bob_bases)

## Step 6: Alice and Bob make their bases public, compare them, and only keep
#          the measurements where they used the same bases.
bob_key = remove_garbage(alice_bases, bob_bases, bob_results)
alice_key = remove_garbage(alice_bases, bob_bases, alice_bits)


## Step 5
sample_size = 15
bit_selection = randint(n, size=sample_size)
bob_sample = sample_bits(bob_key, bit_selection)
print("  bob_sample = " + str(bob_sample))
alice_sample = sample_bits(alice_key, bit_selection)
print("alice_sample = "+ str(alice_sample))

diff_sample = [0 if a == b else 1 for a, b in zip(alice_sample, bob_sample)]
print("      errors = "+ str(diff_sample))

  bob_sample = [1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1]
alice_sample = [1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
      errors = [0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1]


## 3. BB84 in real life

Although this QKD protocol seems perfect at first glance, as with many theoretical concepts, the real world implementation has its problems.

One of the issues with realizing this protocol in real life is that Alice will most likely send Bob multiple photons/qubits for each bit. This is because it is hard to physically create a single photon with 100% reliability.<br>
Such a weakness can be exploited by Eve to obtain information on the shared key ***without*** the knowledge of Alice and Bob. She can achieve this by measuring the additional qubits sent by Alice, which does not affect Bob's measurements.

This method that Eve can use is called the photon-number splitting attack, or PNS attack for short.

## 4. Solving the PNS attack issue: BBM92 protocol

To solve this issue with BB84, we can use a very similar protocol called BBM92. This protocol works in exactly the same way, except for steps 1 and 2.

 ### Step 1

Instead of Alice sending Bob a quantum message, both of them will receive one photon/qubit of an entangled pair.<br>
As you know, when qubits are entangled, we can no longer describe them individually, and we will have to describe both qubits with a single 2-qubit state as follows:

$$\ket{q_A, q_B} = \frac{1}{\sqrt{2}}(\ket{00}+\ket{11})$$

This state will always be measured by Alice and Bob individually to be either $\ket{0}$ or $\ket{1}$ with 50% chance. But it has a particularity: even though the outcome is random, Alice and Bob will always have the exact same measurement outcome, i.e., if Alice measures $\ket{0}$ in the Z-basis, Bob will certainly measure $\ket{0}$ in the Z-basis! And the same goes for $\ket{1}$, $\ket{+}$ and $\ket{-}$.

### Step 2

The difference here is minor: where only Bob was doing the measurement in BB84, both Alice and Bob will need to measure their received qubits. They still however will use different random bases.

The rest of the protocol is exactly the same as BB84.